# Übung

## Übung: Optimization

In [1]:
!pip install sklearn

Pakete importieren

In [2]:
import pandas as pd
import numpy as np
import sklearn as sklearn
import pickle
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

Einlesen der Datensets

In [3]:
with open('../output/bikebuyers/datasets.pkl', 'rb') as handle:
    datasets = pickle.load(handle)
    
print('Datensets geladen')

Datensets geladen


### Baseline erstellen

#### Task 1: Transformer Pipeline laden

Lesen Sie die gespeicherte Transformer Pipeline aus der pickle-Datei '../output/bikebuyers/transformer_pipeline.pkl' und speichern die Pipeline in einer Variable namens transformer_pipeline.

In [4]:
# Hier den Code eingeben

````{Dropdown} Lösung Task 1

  ```{code-block} python
  with open('../output/bikebuyers/transformer_pipeline.pkl', 'rb') as handle:
    transformer_pipeline = pickle.load(handle)
  ```
````

#### Task 2: Pipeline mit Klassifikator erstellen

Erstellen Sie die finale Pipeline, bestehend aus der Transformer Pipeline und anschließendem Predictor in Form eines Entscheidungsbaum-Klassifikator. Verwenden Sie hierzu den [DecisionTreeClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html) von Scikit-learn. Belegen Sie den random_state Parameter mit 0 und verwenden sonst die Standard-Einstellungen. Speichern Sie die Pipeline in einer Variable namens 'full_pipeline'.

In [5]:
# Hier den Code eingeben.

````{Dropdown} Lösung Task 2

  ```{code-block} python
  from sklearn.tree import DecisionTreeClassifier
  full_pipeline = Pipeline(steps=[
    ('transformers', transformer_pipeline),
    ('predictor', DecisionTreeClassifier(random_state=0))
  ])

  ```
````

#### Task 3: Pipeline verwenden

* Trainieren Sie die Pipeline mit dem Trainingsdatenset durch aufrufen der fit()-Methode. 
* Evaluieren Sie das Modell mit dem Validierungsdatenset durch aufrufen der score()-Methode.

Welches Ergebnis erhalten Sie?

In [6]:
# Hier den Code eingeben.

````{Dropdown} Lösung Task 3

  ```{code-block} python
  full_pipeline.fit(datasets['X_train'], datasets['y_train'])
  full_pipeline.score(datasets['X_val'], datasets['y_val'])

  ```
````

### Parameter optimieren

#### Task 4: Grid Search vorbereiten

Erstellen Sie eine Instanz der Klasse [GridSearchCV](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) aus Scikit Learn. Verwenden Sie folgende Paramter-Einstellungen:
* estimator:full_pipeline
* param_grid: 
    * factor-Werte: [1.0, 1.5, 2.0, 3.0]
    * min_samples_split-Werte: [2,3,4,5,6]
* cv: 10

In [7]:
# Hier den Code eingeben.

````{Dropdown} Lösung Task 4

  ```{code-block} python
  from sklearn.model_selection import GridSearchCV

  param_grid = {
    'transformers__num__outlier_remover__factor': [1.0, 1.5, 2.0, 3.0],
    'predictor__min_samples_split': [2,3,4,5,6]
  }

  grid_search = GridSearchCV(full_pipeline, param_grid, cv=10)

  ```
````

#### Task 5: Grid Search anwenden

* Rufen Sie die fit()-Methode unter Verwendung der Trainingsdatensets auf.
* Geben Sie die beste Parameterkombination aus.
* Geben Sie das Ergebnis der besten Parameterkombination aus.

In [8]:
# Hier den Code eingeben

````{Dropdown} Lösung Task 5

  ```{code-block} python
  print("Grid search trainieren ...")
  grid_search.fit(datasets['X_train'], datasets['y_train'])

  print("Best params:")
  print(grid_search.best_params_)
    
  print("Ergebnis mit der besten Parametereinstellung auf den Trainingsdaten:")
  print(f"{grid_search.best_score_:.3f}")
  ```
````

### Merkmale optimieren

#### Task 6: Einkommen und Alter diskretisieren

Erstellen Sie jeweils einen Transformer um die Werte der Merkmale Einkommen und Alter zu diskretisieren. Verwenden Sie folgende Einstellungen:

* Bin-Grenzwerte für das Einkommen: [0, 30000, 60000, 75000, 100000, 150000, 200000]  
* Bin-Grenzwerte für das Alter: [0, 20, 30, 40, 60, 70, 100]

In [9]:
# Hier den Code eingeben.

```{Tip}
* Sie können sich bei der Implementierung am AgeBinned Transformer aus dem Abschnitt Merkmale Optimieren orientieren.
* Das Merkmal Einkommen steht im Pandas Dataframe in der ersten Spalte, der Index beträgt demzufolge 0.  
* Das Merkmal Alter steht im Panas Dataframe in der zweiten Spalte, der Index beträgt demzufolge 1.  
```

````{Dropdown} Lösung Task 6

  ```{code-block} python
  from sklearn.base import BaseEstimator, TransformerMixin

  income_ix, age_ix = 0, 1
  
  class IncomeBinned(BaseEstimator, TransformerMixin):
      def __init__(self):
          pass

      def fit(self, X, y=None):
          return self

      def transform(self, X, y=None):
          X_ = pd.DataFrame(X)
          bins = [0, 30000, 60000, 75000, 100000, 150000, 200000]
          labels = [1,2,3,4,5,6]
          X_[income_ix] = pd.cut(X_[income_ix], bins=bins, labels=labels)
          return X_.values

  class AgeBinned(BaseEstimator, TransformerMixin):
      def __init__(self):
          pass

      def fit(self, X, y=None):
          return self

      def transform(self, X, y=None):
          X_ = pd.DataFrame(X)
          bins = [0, 20, 30, 40, 60, 70, 100]
          labels = [1,2,3,4,5,6]
          X_[age_ix] = pd.cut(X_[age_ix], bins=bins, labels=labels)
          return X_.values
  ```
````

#### Task 7: Transformationen hinzufügen

Übernehmen Sie die optimalen Parameter für folgende Parameter aus Task 5  
* factor des OutlierRemoverExtended-Transformer
* min_samples_split des Prediktors

Fügen Sie drei Schritte in die numerische Pipeline ein:
1. IncomeBinned-Transformer
2. AgeBinned-Transformer
3. StandardScaler-Transformer (von Scikit-learn)

Kopieren Sie hierzu den Code aus der Vorlage und passen Sie die markierten Stellen <mark>[?]</mark> an.

In [10]:
# Fügen Sie den Code aus der Vorlage hier ein und ergänzen Sie die mit [?] markierten Stellen.

````{Dropdown} Code Vorlage 7

  ```{code-block} python
  :emphasize-lines: 9,10,11,12,51
  
  from transformer_extended import OutlierRemoverExtended
  from sklearn.preprocessing import StandardScaler, OneHotEncoder
  from sklearn.pipeline import Pipeline
  from sklearn.compose import ColumnTransformer
  from transformer import OutlierRemover

  # Pipeline für numerische Daten erstellen
  pipeline_numerical = Pipeline(steps=[
    ('outlier_remover', OutlierRemoverExtended(factor=[?])),
    [?],
    [?],
    [?]
  ])

  # Pipeline für kategorische Daten erstellen
  pipeline_categorical = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
  ])

  features_numerical = ['Income', 'Age', 'Cars', 'Children']
  features_categorical = [
    'Marital Status', 
    'Gender', 
    'Education', 
    'Occupation', 
    'Home Owner', 
    'Commute Distance',
    'Region'
  ]

  # Pipelines koordinieren
  transformer_pipeline = ColumnTransformer(
    transformers = [
      (
        'num', 
        pipeline_numerical,
        features_numerical
      ),
      (
        'cat', 
        pipeline_categorical,
        features_categorical
      )
    ])


  full_pipeline_fe1 = Pipeline(steps=[
      ('transformers', transformer_pipeline),
      ('predictor', DecisionTreeClassifier(
        random_state=0,
        min_samples_split=[?]
      ))
  ])

  # Trainieren der Pipeline
  full_pipeline_fe1.fit(datasets['X_train'], datasets['y_train'])
  
  # Ergebnis abfragen
  print("Accuracy-Score nach Hinzufügen der Transformationen:")
  full_pipeline_fe1.score(datasets['X_val'], datasets['y_val'])
    
  ```
````

````{Dropdown} Lösung Task 7

  ```{code-block} python
  :emphasize-lines: 10,11,12,50,51
  
  from transformer_extended import OutlierRemoverExtended
  from sklearn.preprocessing import StandardScaler, OneHotEncoder
  from sklearn.pipeline import Pipeline
  from sklearn.compose import ColumnTransformer
  from transformer import OutlierRemover

  # Pipeline für numerische Daten erstellen
  pipeline_numerical = Pipeline(steps=[
    ('outlier_remover', OutlierRemoverExtended(factor=1.0)),
    ('agebinned', AgeBinned()),
    ('incomebinned', IncomeBinned()),
    ('scaler', StandardScaler())
  ])

  # Pipeline für kategorische Daten erstellen
  pipeline_categorical = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
  ])

  # Numerische und Kategorische Merkmale definieren
  features_numerical = ['Income', 'Age', 'Cars', 'Children']
  features_categorical = [
    'Marital Status', 
    'Gender', 
    'Education', 
    'Occupation', 
    'Home Owner', 
    'Commute Distance',
    'Region'
  ]

  # Pipelines koordinieren
  transformer_pipeline = ColumnTransformer(
    transformers = [
      (
        'num', 
        pipeline_numerical,
        features_numerical
      ),
      (
        'cat', 
        pipeline_categorical,
        features_categorical
      )
    ])

  # Vollständige Pipeline erstellen
  full_pipeline_fe1 = Pipeline(steps=[
      ('transformers', transformer_pipeline),
      ('predictor', DecisionTreeClassifier(
        random_state=0,
        min_samples_split=4
      ))
  ])

  # Trainieren der Pipeline
  full_pipeline_fe1.fit(datasets['X_train'], datasets['y_train'])
  
  # Ergebnis abfragen
  print("Accuracy-Score nach Hinzufügen der Transformationen:")
  full_pipeline_fe1.score(datasets['X_val'], datasets['y_val'])
  ```
````

#### Task 8: Testen der optimierten Pipeline

1. Testen Sie die optimierte Pipeline aus Task 7 mit dem Testdatenset.
2. Vergleichen Sie das Ergebnis mit dem Ergebnis aus Task 7.
3. Vergleichen Sie das Ergebnis mit dem Ergebnis aus Task 3 (Baseline)

In [11]:
# Hier den Code eingeben

In [12]:
# Hier den Code eingeben

````{Dropdown} Lösung Task 8

  ```{code-block} python
  # Erste Code-Zelle
  full_pipeline_fe1.score(datasets['X_val'], datasets['y_val'])
  
  # Zweite Code-Zelle
  full_pipeline_fe1.score(datasets['X_test'], datasets['y_test'])
  ```
  
  1. Der Accuracy-Score beträgt 0.69
  2. Das Ergebnis hat sich nicht verändert. Es zeigt, dass die Pipeline gut generalisiert.
  3. Das Ergebnis hat sich um 0.1 verbessert. Die Verbesserung gegenüber der Baseline ist signifikant.
  
````

#### Task 9: Optimierte Pipeline speichern

Speichern Sie die optimierte Pipeline in einer Pickle-Datei unter '../output/bikebuyers/pipeline.pkl'.

In [13]:
# Hier den Code eingeben.

````{Dropdown} Lösung Task 9

  ```{code-block} python
  with open('../output/bikebuyers/pipeline.pkl', 'wb') as handle:
    pickle.dump(full_pipeline_fe1, handle)
  ```
````

### Lösungen Teil 3: Parameter und Merkmale optimieren

In [14]:
# Code Zelle zum testen.

````{Dropdown} Lösungen kompakt

  ```{code-block} python
  import pickle
  from sklearn.pipeline import Pipeline
  from sklearn.model_selection import GridSearchCV
  from sklearn.tree import DecisionTreeClassifier

  with open('../output/bikebuyers/datasets.pkl', 'rb') as handle:
      datasets = pickle.load(handle)
  print('Dataset geladen')

  with open('../output/bikebuyers/transformer_pipeline.pkl', 'rb') as handle:
      transformer_pipeline = pickle.load(handle)

  full_pipeline = Pipeline(steps=[
      ('transformers', transformer_pipeline),
      ('predictor', DecisionTreeClassifier(random_state=0))
  ])

  param_grid = {
      'transformers__num__outlier_remover__factor': [1.0, 1.5, 2.0, 3.0],
      'predictor__min_samples_split': [2,3,4,5,6],
  }

  grid_search = GridSearchCV(full_pipeline, param_grid, cv=10)
  grid_search.fit(datasets['X_train'], datasets['y_train'])

  print("\nBest params:")
  print(grid_search.best_params_)

  print("\nErgebnis mit der besten Parametereinstellung auf den Trainingsdaten:")
  print(f"{grid_search.best_score_:.3f}")

  best_pipeline = grid_search.best_estimator_

  with open('../output/bikebuyers/full_pipeline.pkl', 'wb') as handle:
      pickle.dump(best_pipeline, handle)
  ```
````

In [15]:
# Code Zelle zum testen.

````{Dropdown} Lösungen kompakt

  ```{code-block} python
  import pandas as pd
  import numpy as np
  from sklearn.base import BaseEstimator, TransformerMixin

  age_ix = 1
  income_ix = 0

  class AgeBinned(BaseEstimator, TransformerMixin):
      def __init__(self):
          pass

      def fit(self, X, y=None):
          return self

      def transform(self, X, y=None):
          X_ = pd.DataFrame(X)
          bins = [0, 20, 30, 40, 60, 70, 100]
          labels = [1,2,3,4,5,6]
          X_[age_ix] = pd.cut(X_[age_ix], bins=bins, labels=labels)
          return X_.values

  class IncomeBinned(BaseEstimator, TransformerMixin):
      def __init__(self):
          pass

      def fit(self, X, y=None):
          return self

      def transform(self, X, y=None):
          X_ = pd.DataFrame(X)
          bins = [0, 30000, 60000, 75000, 100000, 150000, 200000]
          labels = [1,2,3,4,5,6]
          X_[income_ix] = pd.cut(X_[income_ix], bins=bins, labels=labels)
          return X_.values
  ```
````

In [16]:
# Code Zelle zum testen.

````{Dropdown} Lösungen kompakt

  ```{code-block} python
  from transformer_extended import OutlierRemoverExtended
  from sklearn.preprocessing import StandardScaler, OneHotEncoder
  from sklearn.pipeline import Pipeline
  from sklearn.compose import ColumnTransformer
  from transformer import OutlierRemover

  # Pipeline für numerische Daten erstellen
  pipeline_numerical = Pipeline(steps=[
    ('outlier_remover', OutlierRemoverExtended(factor=1.0)),
    ('agebinned', AgeBinned()),
    ('incomebinned', IncomeBinned()),
    ('scaler', StandardScaler())
  ])

  # Pipeline für kategorische Daten erstellen
  pipeline_categorical = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
  ])

  features_numerical = ['Income', 'Age', 'Cars', 'Children']
  features_categorical = [
    'Marital Status', 
    'Gender', 
    'Education', 
    'Occupation', 
    'Home Owner', 
    'Commute Distance',
    'Region'
  ]

  # Pipelines koordinieren
  transformer_pipeline = ColumnTransformer(
    transformers = [
      (
        'num', 
        pipeline_numerical,
        features_numerical
      ),
      (
        'cat', 
        pipeline_categorical,
        features_categorical
      )
    ])


  full_pipeline_fe1 = Pipeline(steps=[
      ('transformers', transformer_pipeline),
      ('predictor', DecisionTreeClassifier(
        random_state=0,
        min_samples_split=4
      ))
  ])

  full_pipeline_fe1.fit(datasets['X_train'], datasets['y_train'])
  full_pipeline_fe1.score(datasets['X_val'], datasets['y_val'])
  ```
````

In [21]:
import pickle
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier

with open('../output/bikebuyers/datasets.pkl', 'rb') as handle:
    datasets = pickle.load(handle)
print('Datensets geladen.')

with open('../output/bikebuyers/transformer_pipeline.pkl', 'rb') as handle:
    transformer_pipeline = pickle.load(handle)
print('Transformer Pipeline geladen.')

full_pipeline = Pipeline(steps=[
    ('transformers', transformer_pipeline),
    ('predictor', DecisionTreeClassifier(random_state=0))
])

param_grid = {
    'transformers__num__outlier_remover__factor': [1.0, 1.5, 2.0, 3.0],
    'predictor__min_samples_split': [2,3,4,5,6],
}

grid_search = GridSearchCV(full_pipeline, param_grid, cv=10)
grid_search.fit(datasets['X_train'], datasets['y_train'])

print("\nBest params:")
print(grid_search.best_params_)

print("\nErgebnis mit der besten Parametereinstellung auf den Trainingsdaten:")
print(f"{grid_search.best_score_:.3f}")

best_pipeline = grid_search.best_estimator_

with open('../output/bikebuyers/full_pipeline.pkl', 'wb') as handle:
    pickle.dump(best_pipeline, handle)

Datensets geladen.
Transformer Pipeline geladen.

Best params:
{'predictor__min_samples_split': 4, 'transformers__num__outlier_remover__factor': 1.0}


Ergebnis mit der besten Parametereinstellung auf den Trainingsdaten:
0.633


In [19]:
import pandas as pd
import numpy as np
from sklearn.base import BaseEstimator, TransformerMixin

age_ix = 1
income_ix = 0

class AgeBinned(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        X_ = pd.DataFrame(X)
        bins = [0, 20, 30, 40, 60, 70, 100]
        labels = [1,2,3,4,5,6]
        X_[age_ix] = pd.cut(X_[age_ix], bins=bins, labels=labels)
        return X_.values
      
class IncomeBinned(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X, y=None):
        X_ = pd.DataFrame(X)
        bins = [0, 30000, 60000, 75000, 100000, 150000, 200000]
        labels = [1,2,3,4,5,6]
        X_[income_ix] = pd.cut(X_[income_ix], bins=bins, labels=labels)
        return X_.values

In [ ]:
from transformer_extended import OutlierRemoverExtended
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from transformer import OutlierRemover

# Pipeline für numerische Daten erstellen
pipeline_numerical = Pipeline(steps=[
  ('outlier_remover', OutlierRemoverExtended(factor=1.0)),
  ('agebinned', AgeBinned()),
  ('incomebinned', IncomeBinned()),
  ('scaler', StandardScaler())
])

# Pipeline für kategorische Daten erstellen
pipeline_categorical = Pipeline(steps=[
  ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

features_numerical = ['Income', 'Age', 'Cars', 'Children']
features_categorical = [
  'Marital Status', 
  'Gender', 
  'Education', 
  'Occupation', 
  'Home Owner', 
  'Commute Distance',
  'Region'
]

# Pipelines koordinieren
transformer_pipeline = ColumnTransformer(
  transformers = [
    (
      'num', 
      pipeline_numerical,
      features_numerical
    ),
    (
      'cat', 
      pipeline_categorical,
      features_categorical
    )
  ])


full_pipeline_fe1 = Pipeline(steps=[
    ('transformers', transformer_pipeline),
    ('predictor', DecisionTreeClassifier(
      random_state=0,
      min_samples_split=4
    ))
])

full_pipeline_fe1.fit(datasets['X_train'], datasets['y_train'])
full_pipeline_fe1.score(datasets['X_val'], datasets['y_val'])